# Objetivo 2.1

# Autores
David Rodríguez , Alejandro Meza

# Índice de la práctica

- [1.Introducción](#1.Introducción)
- [2.Creación del dataset](#2.Creación-del-dataset)
- [3.Creación del clasificador detección de señal](#3.Creación-del-clasificador-detección-de-señal)
- [4.Creación del clasificador distinción de señal](#4.Creación-del-clasificador-distinción-de-señal)
- [5.Predictor Final](#5.Predictor-Final)

In [1]:
import cv2
import glob
import skimage


#Librerias contornos 
import numpy as np
from skimage.io import imread, imshow
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.feature import hog
from skimage import data, exposure

#Machine Learning
from scipy.optimize import minimize
import sklearn.metrics as metrics
from sklearn.model_selection import GridSearchCV
from sklearn import svm

# 1.Introducción

El tercer objetivo ponemos en común los dos objetivos anteriores y los unimos:
-    Objetivo2 
-    Objetivo1
-    Producto final

# 2.Creación de datos

Se va a crear la matriz X, que es la que contiene las imágenes (se crea un dataset). Creamos metodos que nos ayudaran a conseguir nuestro objetivo.

In [2]:
'''
Descripcion: diccionario con los posibles valores de las imagenes, en relacion con el nombre de las carpetas.
Entrada: -
Salida: diccionario cuyas claves son el nombre de las carpetas, y los valores son el tipo de signals que contiene.
'''
def devolverDiccionarioEtiquetasSign():
    
    diccionarioValores = {}
    diccionarioValores['00000'] = 'VelocidadMaxima20'
    diccionarioValores['00001'] = 'VelocidadMaxima30'
    diccionarioValores['00002'] = 'VelocidadMaxima50'
    diccionarioValores['00003'] = 'VelocidadMaxima60'
    diccionarioValores['00004'] = 'VelocidadMaxima70'
    diccionarioValores['00005'] = 'VelocidadMaxima80'
    diccionarioValores['00006'] = 'FinVelocidadMaxima80'
    diccionarioValores['00007'] = 'VelocidadMaxima100'
    diccionarioValores['00008'] = 'VelocidadMaxima120'
    diccionarioValores['00009'] = 'AdelantamientoProhibido'
    diccionarioValores['00010'] = 'AdelantamientoProhibidoParaCamiones'

    diccionarioValores['00011'] = 'InterseccionConPrioridad'
    diccionarioValores['00012'] = 'CalzadaConPrioridad'
    diccionarioValores['00013'] = 'CedaElPaso'
    diccionarioValores['00014'] = 'DetencionObligatoria'
    diccionarioValores['00015'] = 'CirculacionProhibida'
    diccionarioValores['00016'] = 'EntradaProhibidaAVehiculosMercancias'
    diccionarioValores['00017'] = 'EntradaProhibida'
    diccionarioValores['00018'] = 'OtrosPeligros'
    diccionarioValores['00019'] = 'CurvaPeligrosaHaciaLaIzquierda'
    diccionarioValores['00020'] = 'CurvaPeligrosaHaciaLaDerecha'

    diccionarioValores['00021'] = 'CurvasPeligrosasHaciaLaIzquierda'
    diccionarioValores['00022'] = 'PerfilIrregular'
    diccionarioValores['00023'] = 'PavimentoDeslizante'
    diccionarioValores['00024'] = 'EstrechamientoCalzadaPorDerecha'
    diccionarioValores['00025'] = 'Obras'
    diccionarioValores['00026'] = 'Semaforos'
    diccionarioValores['00027'] = 'PasoDePeatones'
    diccionarioValores['00028'] = 'Kids'
    diccionarioValores['00029'] = 'EntradaProhibidasCiclos'
    diccionarioValores['00030'] = 'PavimentoDeslizanteNieveHielo'

    diccionarioValores['00031'] = 'PasoDeAnimalesEnLibertad'
    diccionarioValores['00032'] = 'FinDeProhibiciones'
    diccionarioValores['00033'] = 'SentidoObligatorioDerecha'
    diccionarioValores['00034'] = 'SentidoObligatorioIzquierda'
    diccionarioValores['00035'] = 'SentidoObligatorio'
    diccionarioValores['00036'] = 'DirPermitidasRectoYDerecha'
    diccionarioValores['00037'] = 'DirPermitidasRectoEIzquierda'
    diccionarioValores['00038'] = 'PasoObligatorioDerecha'
    diccionarioValores['00039'] = 'PasoObligatorioizquierda'
    diccionarioValores['00040'] = 'InterseccionSentidoObligatorioGiratorio'


    diccionarioValores['00041'] = 'FinProhibicionAdelantamiento'
    diccionarioValores['00042'] = 'FinProhibicionAdelantamientoCamiones'
    
    return diccionarioValores

In [3]:
'''
Descripcion: diccionario con los posibles valores de las imagenes, en relacion con el nombre de las carpetas.
Entrada: -
Salida: diccionario cuyas claves son el nombre de las carpetas, y los valores son el tipo de signals que contiene.
'''
def devolverDiccionarioEtiquetasNoSign():
    
    diccionarioValores = {}
    diccionarioValores['000'] = 'Animal'
    diccionarioValores['001'] = 'Carretera'
    diccionarioValores['002'] = 'Cielo'
    diccionarioValores['003'] = 'VehiculoCoche'
    diccionarioValores['004'] = 'Arbol'
    diccionarioValores['005'] = 'Persona'
    diccionarioValores['006'] = 'Edificio'
    diccionarioValores['007'] = 'VehiculoMoto'
    diccionarioValores['008'] = 'Semaforo'
    diccionarioValores['009'] = 'Basura'
    diccionarioValores['010'] = 'Farola'

    diccionarioValores['011'] = 'Estatua'
    diccionarioValores['012'] = 'PasoDeZebra'
    diccionarioValores['013'] = 'Carteles'
    diccionarioValores['014'] = 'Ventana'
    diccionarioValores['015'] = 'Paisaje'
    diccionarioValores['016'] = 'Mix'
    diccionarioValores['017'] = 'All'
    diccionarioValores['018'] = 'Prueba'
    
    return diccionarioValores

Creamos un diccionario para almacenar todas las fotos, relacionadas a cada elemento diferente de señal

In [4]:
'''
Descripcion: diccionario con los posibles valores de las imagenes, en relacion con el nombre de las carpetas.
Entrada: -
Salida: diccionario cuyas claves son el nombre de las carpetas, y los valores son las imagenes de las signals
'''
def crearDiccionarioDatosSign():
    trafic_signals = {}
    #para cada tipo de señal creamos una lista con sus ejemplos, y se la asignamos a su etiqueta de nombre de carpeta
    for i in range(0,43):
        if i<10:
            folder = 'Dataset_traffic_sign/0000'+str(i)+'/*.png'
            tipo = '0000'+str(i)
        else:
            folder = 'Dataset_traffic_sign/000'+str(i)+'/*.png'
            tipo = '000'+str(i)
        
        images = [cv2.imread(file,0) for file in glob.glob(folder)]
        images_mod = [] 
        for im in images:
            #se aplica el resize
            images_mod.append(cv2.resize(im, (100, 100), interpolation = cv2.INTER_NEAREST))

        trafic_signals[tipo] = images_mod
    return trafic_signals

In [5]:
'''
Descripcion: diccionario con los posibles valores de las imagenes, en relacion con el nombre de las carpetas.
Entrada: -
Salida: diccionario cuyas claves son el nombre de las carpetas, y los valores los elementos que no son signals
'''
def crearDiccionarioDatosNoSign():
    noSignals = {} #diccionario para almacenar todo lo que no es sinal
    #para cada tipo de elemento que no es señal creamos una lista con sus ejemplos
    # y se la asignamos a su etiqueta de nombre de carpeta
    for i in range(0,18): #iteramos sobre el numero de carptas
        if i<10:
            folder = 'Dataset_no_traffic_sign/00'+str(i)+'/*.jpg'
            tipo = '00'+str(i)
        else:
            folder = 'Dataset_no_traffic_sign/0'+str(i)+'/*.jpg'
            tipo = '0'+str(i)

        images = [cv2.imread(file,0) for file in glob.glob(folder)]
        images_mod = [] 
        for im in images:
            #se aplica el resize
            images_mod.append(cv2.resize(im, (100, 100), interpolation = cv2.INTER_NEAREST))

        noSignals[tipo] = images_mod
    return noSignals

Creamos funciones para transformar imagenes en descriptores con HOG.

In [6]:
# HOG
def hogMaker(images00000_mod):
    fd, hog_image = hog(images00000_mod, orientations=8, pixels_per_cell=(16, 16),cells_per_block=(1, 1), visualize=True)
    return fd

Cogemos las imagenes y las separamos en train y test, y decimos a que clase pertenecen*

In [7]:
'''
Descripcion: Obtencion de train y test a partir de un conjunto de imágenes
Entrada: trafic_signals, signals_types
Salida: imagenesTrain,imagenesTest,ytrain,ytest
'''
def separaTrainTest(trafic_signals,signals_types):
    imagenesTrain = None
    imagenesTest = None
    ytrain = None
    ytest = None
    
    for signal,images in trafic_signals.items():
        
        aux_y_train = []
        aux_y_test = []
        
        np_im = np.array(images)
        n_train = int(0.6*np_im.shape[0])
        n_test = np_im.shape[0]-n_train
        
        aux_train = np_im[:n_train]
        aux_test = np_im[n_train:]
        
        for i in range(n_train):
            aux_y_train.append(signals_types[signal])
        for i in range(n_test):
            aux_y_test.append(signals_types[signal])
        
        aux_y_train = np.array(aux_y_train).reshape(-1,1)
        aux_y_test = np.array(aux_y_test).reshape(-1,1)
        
        if imagenesTrain is None:
            imagenesTrain = aux_train
            ytrain = aux_y_train
            imagenesTest = aux_test
            ytest = aux_y_test
        else:
            imagenesTrain = np.concatenate((imagenesTrain,aux_train),axis=0)
            ytrain =  np.vstack((ytrain,aux_y_train))
            imagenesTest = np.concatenate((imagenesTest,aux_test),axis=0)
            ytest = np.vstack((ytest,aux_y_test))
        
    return imagenesTrain,imagenesTest,ytrain,ytest

Creamos las caracteristicas con los datos que nos den y sus tipos*

In [8]:
def creaMatrizCaracteristicas(imagenes,carac_type):
    X = None
        
    for i in range(imagenes.shape[0]):
        if carac_type=='hog':
            d = hogMaker(imagenes[i])
        else: #en caso de recibir un valor que no es valido
            try:
                raise ValueError
            except ValueError:
                print("valor incorrecto")
        
        if d is not None:
            if X is None:
                X = d
            else:
                X = np.vstack((X,d))
    return X

In [9]:
elem_signals = crearDiccionarioDatosSign() #creacion del diccionario de datos
signals_types = devolverDiccionarioEtiquetasSign() #creacion de diccionario que contiene nombre de carpetas y tipos de signals

elem_no_signals = crearDiccionarioDatosNoSign() #creacion del diccionario de datos
no_signals_types = devolverDiccionarioEtiquetasNoSign() #creacion de diccionario que contiene nombre de carpetas y tipos de signalcarac_type = 'hog' #el tipo elegido para los descriptores va a ser Hog
carac_type = 'hog'

In [10]:
signal_Train,signal_Test,signal_ytrain,signal_ytest = separaTrainTest(elem_signals,signals_types)

In [11]:
no_signal_Train,no_signal_Test,no_signal_ytrain,no_signal_ytest = separaTrainTest(elem_no_signals,no_signals_types)

In [12]:
imagenesTrain = np.concatenate((signal_Train, no_signal_Train))
imagenesTest = np.concatenate((signal_Test, no_signal_Test))

Ytrain = np.vstack((signal_ytrain,no_signal_ytrain))
Ytest = np.vstack((signal_ytest,no_signal_ytest))

In [13]:
imagenesTrain_GK = np.zeros(imagenesTrain.shape)
for i in range(imagenesTrain.shape[0]):
    imagenesTrain_GK[i] = cv2.GaussianBlur(imagenesTrain[i],(3,3),cv2.BORDER_REPLICATE)
    
imagenesTest_GK = np.zeros(imagenesTest.shape)
for i in range(imagenesTest.shape[0]):
    imagenesTest_GK[i] = cv2.GaussianBlur(imagenesTest[i],(3,3),cv2.BORDER_REPLICATE)
    
Xtrain = creaMatrizCaracteristicas(imagenesTrain_GK,carac_type)
Xtest = creaMatrizCaracteristicas(imagenesTest_GK,carac_type)

[Indice](#Índice-de-la-práctica)

# 3.Creación del clasificador detección de señal

In [14]:
def transformarY_SignNoSign(y, dicNoSign):
    Y = np.ones(y.shape[0]).reshape(-1,1)
    
    for signal,nombre in dicNoSign.items():
        for i in range(Y.size):
            if y[i]==nombre:
                Y[i] = 0
    Yfinal = []
    for i in range(Y.size):
        if Y[i] == 1:
            Yfinal.append('Signal')
        else:
            Yfinal.append('No Signal')
    return Yfinal

In [15]:
Ytrain_T = transformarY_SignNoSign(Ytrain, no_signals_types)
Ytest_T = transformarY_SignNoSign(Ytest, no_signals_types)

In [16]:
Ytrain_T = np.array(Ytrain_T)
Ytest_T = np.array(Ytest_T)

In [ ]:
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [ 0.001, 0.0001], 'kernel': ['rbf']},]
svc = svm.SVC()
clf = GridSearchCV(svc, param_grid)
clf.fit(Xtrain,Ytrain_T.ravel())
y_pred = clf.predict(Xtest)
print("Classification report for - \n{}:\n{}\n".format(clf, metrics.classification_report(Ytest_T.ravel(), y_pred)))

In [ ]:
def predecirImagenSVM(imagen,carac_type,clf):
#     transformar imagen en des
    d = creaMatrizCaracteristicas(np.array([imagen]), carac_type)
    d = np.array(d).reshape(1,-1)
    return clf.predict(d)

In [ ]:
def sliding_window(image, entreVentanas, windowSize):
    # slide a window across the image
    for j in range(0, image.shape[0], entreVentanas):
        for i in range(0, image.shape[1], entreVentanas):
            # yield the current window
            yield (i, j, image[j:j + windowSize[1], i:i + windowSize[0]])

(La ventana la vamos a implementar en la función final)

[Indice](#Índice-de-la-práctica)

# 4.Creación del clasificador distinción de señal

Primero separamos las imagenes de señales en conjuntos de Train y Test como en el objetivo anterior para entrenar la SVM y para probarla. Y les aplicamos un filtro gaussiano el cual 

In [ ]:
signal_Train,signal_Test,signal_ytrain,signal_ytest = separaTrainTest(elem_signals,signals_types)

In [ ]:
imagenesTrainForSignalRecog,imagenesTestForSignalRecog,Ytrain_Sign,Ytest_Sign = separaTrainTest(elem_signals,signals_types)

In [ ]:
signal_Train_GK = np.zeros(signal_Train.shape)
for i in range(signal_Train.shape[0]):
    signal_Train_GK[i] = cv2.GaussianBlur(signal_Train[i],(3,3),cv2.BORDER_REPLICATE)

signal_Test_GK = np.zeros(signal_Test.shape)
for i in range(signal_Test_GK.shape[0]):
    signal_Test_GK[i] = cv2.GaussianBlur(signal_Test[i],(3,3),cv2.BORDER_REPLICATE)
    
Xtrain_Sign = creaMatrizCaracteristicas(signal_Train_GK,carac_type)
Xtest_Sign = creaMatrizCaracteristicas(signal_Test_GK,carac_type)

Entrenamos la SVM

In [ ]:
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},]
svc = svm.SVC()
clf_signaltype = GridSearchCV(svc, param_grid)
clf_signaltype.fit(Xtrain_Sign, Ytrain_Sign.ravel())
y_pred = clf_signaltype.predict(Xtest_Sign)
print("Classification report for - \n{}:\n{}\n".format(clf_signaltype, metrics.classification_report(Ytest_Sign.ravel(), y_pred)))

[Indice](#Índice-de-la-práctica)

# 5.Predictor Final

En la siguiente función vamos a juntar lo creado anteriormente para que como entrada le pasemos el "path" de la imagen a analizar y nos devuelva la imagen con las señales encontradas y predichas.

In [ ]:
def dondeHaySignal(imagen_nombre,clf,clf_signaltype):
    
    imagen_sin_color = cv2.imread(imagen_nombre,0)
    imagen_color = cv2.imread(imagen_nombre)
    
#     Predecimos que parches son señales
    if imagen_sin_color.shape[0]<imagen_sin_color.shape[1]:
        winH = int(imagen_sin_color.shape[0])
    else:
        winH = int(imagen_sin_color.shape[1])
    winW = winH
    step = int(winW/2)
    result = []
    while winW>30 and winH>30:
        for (x, y, window) in sliding_window(imagen_sin_color, step, windowSize=(winW, winH)):
            if x+winW<=imagen_sin_color.shape[1] and y+winH<=imagen_sin_color.shape[0]:
                res = predecirImagenSVM(cv2.resize(window, (100, 100), interpolation = cv2.INTER_NEAREST),carac_type,clf)
                if res=='Signal':
                    result.append(np.array([x,y,winW,winH]))
        winW = int(winW/1.5)
        winH = winW
        step = int(winW/2)
        
#     Si hay alguna prediccion incluida en otra se elimina la 
#     que está incluida (exclusion de no maximos) en otra
    n_result = []
    repetidos = []
    for i in range(len(result)):
        for j in range(len(result)):
            if result[j][0]>=result[i][0] and result[j][1]>=result[i][1] and result[j][0]+result[j][2]<=result[i][0]+result[i][2] and result[j][1]+result[j][3]<=result[i][1]+result[i][3] and i!=j:
                repetidos.append(j)

    for i in range(len(result)):
        if i not in repetidos:
            n_result.append(result[i])
            
#     Saca la imagen con la señal identificada
    clone = imagen_color.copy()
    for i in range(len(n_result)):
        x,y,winW,winH = n_result[i]
        parche = imagen_sin_color[y:y + winH, x:x + winW]
        res = predecirImagenSVM(cv2.resize(parche, (100, 100), interpolation = cv2.INTER_NEAREST),carac_type,clf_signaltype)
        cv2.rectangle(clone, (x, y), (x + winW, y + winH), (0, 255, 0), 2)
        cv2.putText(clone,str(res), (x,y-10), cv2.FONT_HERSHEY_PLAIN,.7,(0, 255, 0))
        cv2.imshow("Window", clone)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Probamos la funcion anterior con un ejemplo:

In [ ]:
dondeHaySignal('Dataset_images/00001.jpg',clf,clf_signaltype)

[Indice](#Índice-de-la-práctica)

En resumen, como podemos apreciar en este ejemplo, el objetivo ha sido un exito: hemos podido clasificar adecuadamente la posicion y el tipo de señal. 